In [1]:
import sys
import os

# Ruta relativa desde el notebook a la carpeta de scripts
sys.path.append(os.path.abspath("../scripts"))

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv("../datasets/Rest-Mex_2025_train.csv", encoding='utf-8')

In [4]:
train_df.columns

Index(['Title', 'Review', 'Polarity', 'Town', 'Region', 'Type'], dtype='object')

In [8]:
train_df["Town"].value_counts().index.to_numpy()

array(['Tulum', 'Isla_Mujeres', 'San_Cristobal_de_las_Casas',
       'Valladolid', 'Bacalar', 'Palenque', 'Sayulita', 'Valle_de_Bravo',
       'Teotihuacan', 'Loreto', 'TodosSantos', 'Patzcuaro', 'Taxco',
       'Tlaquepaque', 'Ajijic', 'Tequisquiapan', 'Metepec', 'Tepoztlan',
       'Cholula', 'Tequila', 'Orizaba', 'Izamal', 'Creel',
       'Ixtapan_de_la_Sal', 'Zacatlan', 'Huasca_de_Ocampo', 'Mazunte',
       'Xilitla', 'Atlixco', 'Malinalco', 'Bernal', 'Tepotzotlan',
       'Cuetzalan', 'Chiapa_de_Corzo', 'Parras', 'Dolores_Hidalgo',
       'Coatepec', 'Cuatro_Cienegas', 'Real_de_Catorce', 'Tapalpa'],
      dtype=object)

In [9]:
towns = { town:idx for idx, town in enumerate(train_df["Town"].value_counts().index.to_numpy()) }
train_df["label_town"] = train_df["Town"].apply(lambda x: towns[x])

In [13]:
train_df["label_polarity"] = train_df["Polarity"].apply(lambda x: int(x-1))

In [11]:
types = { _type:idx for idx, _type in enumerate(train_df["Type"].value_counts().index.to_numpy()) }
train_df["label_type"] = train_df["Type"].apply(lambda x: types[x] )

In [14]:
train_df.head()

,Title,Review,Polarity,Town,Region,Type,label_town,label_polarity,label_type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant,6,4,0
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive,0,3,1
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel,0,2,2
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive,1,3,1
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel,11,4,2


### Caracters

In [15]:
import numpy as np

characters = np.unique_values([ c for review in train_df["Review"] for c in review ])

In [ ]:
import re
import unicodedata

def clean_characters(text):
    #text = re.sub('\n+', '\n', text)
    text = re.sub(r"`|‘|’|´", "'", text)
    text = re.sub(r"''", "", text)
    text = re.sub(r"\xad|\x81|…|_|\u200b", " ", text)
    text = re.sub("[-—–]+", "-", text) 
    text = re.sub(r"\.+", ".", text)
    
    text = re.sub(r"б|Ã¡|Ã¡|à", "á", text)
    text = re.sub(r"Ã©|è|й", "é", text)
    text = re.sub(r"у|Ã³|í³|ò", "ó", text)
    text = re.sub(r"ъ|Ãº|ù", "ú", text)

    text = re.sub(r"Ã|À", "Á", text)
    text = re.sub(r"Ã‰|È", "É", text)
    text = re.sub(r"Ã|Ì", "Í", text)
    text = re.sub(r"Ã“|Ò", "Ó", text)
    text = re.sub(r"Ãš|Ù", "Ú", text)
    
    text = re.sub(r"Ã‘", "Ñ", text)
    text = re.sub(r"с|Ã±|a±|í±", "ñ", text)
    text = re.sub(r"е", "e", text)
    text = re.sub(r"Â¿|Ї", "¿", text)
    text = re.sub(r"éÂ¼", "üe", text)
    text = re.sub(r"ss ", " ", text)  
    
    if re.search(r"н|Ã ­|Ã|­�|ì", text) is not None: 
        text = re.sub(r"н|Ã ­|Ã|­�|ì", "í", text)
        text = re.sub(r"í ", "í", text)

    text = re.sub(r'([\.,¡!¿\?\[\]\(\)%\$])', r' \1 ', text)
    text = re.sub(r"[^A-Za-zÁÉÍÓÚáéíóúüÑñ¿\?!¡,\[\]\(\)\n'0-9-\. %\$]", '', text)
    
    text = re.sub(r'\b[Mm][áa]s\b[\.\!\?]?$','', text)

    text = re.sub(r"\s+", " ", text)

    return unicodedata.normalize('NFC', text.strip())

In [24]:
#from dataset_utils import clean_characters

train_df["clean_review"] = train_df["Review"].apply(lambda x: clean_characters(x))

In [18]:
new_characters = np.unique_values([ c for review in train_df["clean_review"] for c in review ])
new_characters

array([' ', '!', '$', '%', "'", '(', ')', ',', '-', '.', '0', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E',
       'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
       'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c',
       'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
       'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¿', 'Á',
       'É', 'Í', 'Ñ', 'Ó', 'Ú', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü'],
      dtype='<U1')

In [26]:
# 121199 121196 20450
id = 121196
print(train_df["Review"][id])
print("")
print(train_df["clean_review"][id])

Hay que ver para creer (pero quizÃ¡s no deberÃ­a. . . parte de su encanto es lo tranquilo que es). . . Nuestro nuevo lugar favorito en MÃ©xico. Vistas increÃ­bles, gente amable, deliciosa comida y bebidas, artesanos de talento - todo se Ãºnen para conseguir para una autÃ©ntica fiesta mexicana.

Hay que ver para creer ( pero quizás no debería . . . parte de su encanto es lo tranquilo que es ) . . . Nuestro nuevo lugar favorito en México . Vistas increíbles , gente amable , deliciosa comida y bebidas , artesanos de talento - todo se únen para conseguir para una auténtica fiesta mexicana .


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

words_size = [ len(text.split(" ")) for text in train_df["clean_review"] ]
words_mean = np.mean(words_size)
words_q    = np.quantile(words_size, q=0.95)

In [ ]:
np.quantile(words_size, q=0.8)

In [ ]:
train_df["clean_review"].iloc[np.argmax(words_size)].split(".")[3]

In [ ]:
plt.hist(words_size, bins=np.arange(1,512,1))
plt.axvline(words_mean, color="red", linestyle="dashed", linewidth=2, label=f"Mean: {words_mean:.2f}")
plt.axvline(words_q, color="green", linestyle="dashed", linewidth=2, label=f"Quantile: {words_q:.2f}")
plt.legend()
plt.show()